In [1]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 10
BS = 32

In [3]:
# r before string will prevent Python from interpreting escape characters.
DIRECTORY = r"D:\HopeAI\Week11-Deep Learning Module\Facemask\dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [14]:
#from sklearn.preprocessing import LabelBinarizer
# Initialize LabelBinarizer to encode the labels (if necessary)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)
print(labels)

[]


In [16]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []

[INFO] loading images...


In [9]:
from imutils import paths

# Define the path to your dataset
dataset_path = r"D:\HopeAI\Week11-Deep Learning Module\Facemask\dataset"

# List all image paths in the dataset
imagePaths = list(paths.list_images(dataset_path))

# Print the number of images found and a few example paths
print(f"Found {len(imagePaths)} images.")
print("Example image paths:")
print(imagePaths[:5])  # Print first 5 image paths

Found 3848 images.
Example image paths:
['D:\\HopeAI\\Week11-Deep Learning Module\\Facemask\\dataset\\without_mask\\0.jpg', 'D:\\HopeAI\\Week11-Deep Learning Module\\Facemask\\dataset\\without_mask\\0_0_aidai_0014.jpg', 'D:\\HopeAI\\Week11-Deep Learning Module\\Facemask\\dataset\\without_mask\\0_0_aidai_0029.jpg', 'D:\\HopeAI\\Week11-Deep Learning Module\\Facemask\\dataset\\without_mask\\0_0_aidai_0043.jpg', 'D:\\HopeAI\\Week11-Deep Learning Module\\Facemask\\dataset\\without_mask\\0_0_aidai_0074.jpg']


In [13]:
import warnings
# Ignoring all warnings
warnings.filterwarnings("ignore")

In [17]:
# Loop over the image paths
for imagePath in imagePaths:
    # Extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # Load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    # Update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# Convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

print(data)
print(labels)

[[[[-0.94509804 -0.9764706  -0.85882354]
   [-0.94509804 -0.9764706  -0.85882354]
   [-0.94509804 -0.9764706  -0.85882354]
   ...
   [-0.8509804  -0.90588236 -0.78039217]
   [-0.8509804  -0.90588236 -0.78039217]
   [-0.84313726 -0.8980392  -0.77254903]]

  [[-0.9137255  -0.94509804 -0.827451  ]
   [-0.9137255  -0.94509804 -0.827451  ]
   [-0.9137255  -0.94509804 -0.827451  ]
   ...
   [-0.7490196  -0.8039216  -0.6784314 ]
   [-0.7490196  -0.8039216  -0.6784314 ]
   [-0.7411765  -0.79607844 -0.67058825]]

  [[-0.92156863 -0.9607843  -0.8039216 ]
   [-0.92156863 -0.9529412  -0.81960785]
   [-0.92156863 -0.9607843  -0.8039216 ]
   ...
   [-0.88235295 -0.8980392  -0.7882353 ]
   [-0.85882354 -0.8901961  -0.77254903]
   [-0.85882354 -0.8745098  -0.7647059 ]]

  ...

  [[ 0.38823533 -0.9607843  -0.36470586]
   [ 0.38823533 -0.9607843  -0.3960784 ]
   [ 0.37254906 -0.9529412  -0.44313723]
   ...
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         

In [18]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 20% for testing
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [19]:
# Construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [20]:
# Construct the validation image generator
val_aug = ImageDataGenerator()

In [21]:
# Create data generators
train_generator = aug.flow(trainX, trainY, batch_size=BS)
val_generator = val_aug.flow(testX, testY, batch_size=BS)

In [22]:
train_generator 

In [23]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [24]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [25]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [26]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [53]:
# Compile the model with binary_crossentropy loss 
INIT_LR = 0.001 # Replace with your initial learning rate 
BS = 16 # Batch size 
EPOCHS = 10 # Number of epochs 
opt = Adam(learning_rate=INIT_LR) 
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
  #ValueError: `logits` and `labels` must have the same shape, received ((None, 2) vs (None, 1)).--fit function

In [ ]:
# Compile the model model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [48]:
print("[INFO] training head...")

[INFO] training head...


In [62]:
len(trainX) 

3078

In [63]:
len(testX)

770

In [60]:
labels.shape

(3848,)

In [66]:
# Train the head of the network
H = model.fit(
    train_generator,
    steps_per_epoch=len(trainX) // BS,
    validation_data=val_generator,
    validation_steps=len(testX) // BS,  # Using testX here
    epochs=EPOCHS
)


Epoch 1/10


ValueError: in user code:

    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\hp\anaconda3\envs\vetensor\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
   

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...
32/49 [==================>...........] - ETA: 10s

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

In [ ]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("Face_Mask_Detector.h5")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")